In [1]:
import pandas as pd 
import numpy as np 
import gc
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns 

In [4]:
def trim_categories(df, cols_list):
    for col in cols_list:
        df[col].cat.rename_categories(df[col].cat.categories.astype('str').str.strip(), inplace=True)

In [5]:
transaction_df_cols = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6']
tr_dtypes= {'tr_id' : 'category',  
            'tr_date' : 'uint32', 
            'tr_time' : 'uint32', 
            'tr_station_id' : 'category', 
            'tr_station_name' : 'category', 
            'tr_station_cty' : 'category', 
            'tr_station_st' : 'category', 
            'tr_drvr_code' : 'category', 
            'tr_unit_id' : 'category', 
            'tr_item_cd1' : 'category', 
            'tr_item_qty1' : 'float32', 
            'tr_item_cd2' : 'category', 
            'tr_item_qty2' : 'float32', 
            'tr_item_cd3' : 'category', 
            'tr_item_qty3' : 'float32', 
            'tr_item_cd4' : 'category', 
            'tr_item_qty4' : 'float32', 
            'tr_item_cd5' : 'category', 
            'tr_item_qty5' : 'float32', 
            'tr_item_cd6' : 'category', 
            'tr_item_qty6' : 'float32'}
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
transaction_df = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv', names = transaction_df_cols, dtype = tr_dtypes, header=0 )

# Trim white space from category strings 
trim_categories(transaction_df, ['tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st','tr_drvr_code','tr_unit_id','tr_item_cd1', 'tr_item_cd2', 'tr_item_cd3', 'tr_item_cd4', 'tr_item_cd5', 'tr_item_cd6' ])
transaction_df['tr_id'] = transaction_df.tr_id.cat.as_ordered()

In [6]:
#confirm no junk data in this col (returns false if no NaNs)
None in transaction_df.tr_time

# 20170112 3

#Combine tr_date and tr_time into a single date time column
    #1. zero pad tr_time on left for interpretation as military time
        #cast tr_time as string
transaction_df.tr_time = transaction_df.tr_time.astype(str)

# 20170112 '3' Time column is now a string
#ensure none of my strings are over 4 digits in length
for s in transaction_df.tr_time:
    assert len(s) < 5

#check for blanks
'' in transaction_df.tr_time

#add zeros to the left until each cell is len 4
zero_padded_time = []
for s in transaction_df.tr_time:
    while len(s) < 4:
        s = '0' + s
    zero_padded_time.append(s)
# 20170112 '0003' zero padded minutes
#spot check
for s in zero_padded_time:
    assert len(s) == 4

transaction_df.tr_time = zero_padded_time

#append tr_date (as string) and tr_time together, and then convert them to dateime
#transaction_df.tr_datetime 
tst = [datetime.strptime(str(d) + 
                         t, '%Y%m%d%H%M') 
       for d, t in zip(transaction_df.tr_date,
                       transaction_df.tr_time)]
#love list comprehensions

transaction_df['tr_datetime'] = tst#assign

#drop superfluous columns
transaction_df.drop(['tr_date','tr_time'], axis = 'columns', inplace=True)
transaction_df.head()

,tr_id,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,1.000000,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 00:20:00
1,42575331,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,137.399994,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0.0,2017-01-01 00:48:00
2,42578678,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,137.929993,DEFD,4.12,,0.0,,0.0,,0.0,,0.0,2017-01-01 01:34:00
3,42578722,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,45.000000,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 01:37:00
4,42579695,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,107.360001,,0.00,,0.0,,0.0,,0.0,,0.0,2017-01-01 03:02:00


In [54]:
fuel_trans_df = transaction_df[transaction_df.tr_item_cd1.isin(act_fuel_transactions)]

In [55]:
perf_cols = ['Id', 'EventTimeStamp', 'EquipmentID', 'DriverID', 'IgnitionStatus', 'TripStatus',
       'DataStartTimeStamp', 'DataEndTimeStamp', 'Distance', 'DriveTime',
       'EngineTime', 'MoveTime', 'IntertripIdleTime', 'ShortIdleTime',
       'OverSpeedTime', 'TotalFuelUsed', 'IdleFuelUsed',
        'CruiseControlTime', 'TopGearTime',
       'GearDataSource', 'SpeedUnits', 'LocationTimeStamp', 'Latitude', 'Longitude', 
            'FaultFlag', 'RegisteredDriver']

perf_col_types = {'DriverID': 'category',
             'EFReportReason': 'category',
             'Id': 'uint64',
             'IgnitionStatus': 'category',
             'GearDataSource': 'category',
             'Latitude': 'float64',
             'Longitude': 'float64',
             'FaultFlag': 'category',
             'RegisteredDriver': 'category'}

performance_df = pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv',
                             usecols=perf_cols, 
                            dtype=perf_col_types, 
                            parse_dates=['EventTimeStamp','LocationTimeStamp', 'DataStartTimeStamp','DataEndTimeStamp' ], 
                            infer_datetime_format=True)
# Remove invalid Truck IDs
performance_df = performance_df[performance_df.EquipmentID < 9999]

# Set EquipmentID to category 
performance_df['EquipmentID'] = performance_df.EquipmentID.astype('category')

# Trim white space from category strings 

trim_categories(performance_df, ['EquipmentID', 'DriverID'])

In [2]:
dtype_dict = {
    'Id' : 'uint64', 
    'EquipmentID' : 'category', 
    'DriverID' : 'category', 
    'Latitude' : 'float64', 
    'Longitude' : 'float64', 
    'Speed' : 'uint16', 
    'Odometer' : 'float64', 
    'IgnitionStatus' : 'category', 
    'EFReportReason' : 'category', 
    'TankLevelPercent' : 'float32', 
    'TankLevelGallons' : 'float32', 
    'ExactFuelEventId' : 'uint64', 
    'truck' : 'category', 
    'tankcap' : 'uint16', 
    'refuel_tr_sum' : 'float32', 
    'refuel_tr_id' : 'category' }
col_names = ['idx1', 'idx2', 'Id', 'EventTimeStamp', 'EquipmentID', 'DriverID', 'Latitude',
       'Longitude', 'LocationTimeStamp', 'Speed', 'Odometer', 'IgnitionStatus',
       'EFReportReason', 'TankLevelPercent', 'TankLevelGallons',
       'ExactFuelEventId', 'truck', 'tankcap', 'refuel_datetime',
       'refuel_tr_sum', 'refuel_tr_id']
date_cols = ['EventTimeStamp', 'LocationTimeStamp', 'refuel_datetime' ]

In [3]:
events_levels_df = pd.read_csv('big_files/events_levels_df.gz', skiprows=1, names=col_names,
                               dtype=dtype_dict, usecols=np.arange(2,21), 
                              parse_dates = date_cols, infer_datetime_format = True)

In [ ]:
]

In [9]:
events_levels_df.head()

,Id,EventTimeStamp,EquipmentID,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Odometer,IgnitionStatus,EFReportReason,TankLevelPercent,TankLevelGallons,ExactFuelEventId,truck,tankcap,refuel_datetime,refuel_tr_sum,refuel_tr_id
0,12649676,2017-01-01 00:19:45.357,1559,NaN,36.06712,-86.43476,2017-01-01 00:01:05,0,505629.3,1,2,96.0,240.0,12649676,1559,250,NaT,NaN,NaN
1,12649677,2017-01-01 00:34:45.370,1559,NaN,36.06712,-86.43476,2017-01-01 00:01:05,0,505629.3,1,2,96.0,240.0,12649677,1559,250,NaT,NaN,NaN
2,12649678,2017-01-01 00:49:45.417,1559,NaN,36.06712,-86.43476,2017-01-01 00:01:05,0,505629.3,1,2,96.0,240.0,12649678,1559,250,NaT,NaN,NaN
3,12649679,2017-01-01 01:04:45.463,1559,NaN,36.06712,-86.43476,2017-01-01 00:01:05,0,505629.3,1,2,96.0,240.0,12649679,1559,250,NaT,NaN,NaN
4,12649680,2017-01-01 01:19:45.480,1559,NaN,36.06712,-86.43476,2017-01-01 00:01:05,0,505629.3,1,2,96.0,240.0,12649680,1559,250,NaT,NaN,NaN


In [13]:
events_levels_df = events_levels_df.sort_values(['truck', 'EventTimeStamp']).reset_index()
events_levels_df.info(memory_usage='deep', null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7908406 entries, 0 to 7908405
Data columns (total 20 columns):
index                7908406 non-null int64
Id                   7908406 non-null uint64
EventTimeStamp       7908406 non-null datetime64[ns]
EquipmentID          7908406 non-null category
DriverID             7798523 non-null category
Latitude             7908406 non-null float64
Longitude            7908406 non-null float64
LocationTimeStamp    7908406 non-null datetime64[ns]
Speed                7908406 non-null uint16
Odometer             7908406 non-null float64
IgnitionStatus       7908406 non-null category
EFReportReason       7908406 non-null category
TankLevelPercent     7908406 non-null float32
TankLevelGallons     7908406 non-null float32
ExactFuelEventId     7908406 non-null uint64
truck                7908406 non-null category
tankcap              7908406 non-null uint16
refuel_datetime      7795833 non-null datetime64[ns]
refuel_tr_sum        7795833 non-null f

In [ ]:
bad_ids = pd.read_csv('big_files/events_flags.csv', header=None, names=['events_flags'])

In [14]:
bad_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7908406 entries, 0 to 7908405
Data columns (total 1 columns):
events_flags    bool
dtypes: bool(1)
memory usage: 7.5 MB


In [16]:
events_levels_df[bad_ids.events_flags].shape

(4910167, 20)

In [56]:
#extracting the transactions details from fuel transactions df
grp_transactions = pd.DataFrame({'Total_Fuel_Purchased' :
                         fuel_trans_df.groupby(['tr_drvr_code','tr_unit_id'])['tr_item_qty1'].sum()
                        }).reset_index()

#extracting the min and max odometer reading for every truck and driver combo
grp_data_events = pd.DataFrame(events_levels_df.groupby(['DriverID','EquipmentID'])['Odometer'].agg({'min','max'}).add_prefix('odo_')).reset_index()

#triming the spaces 
trim_categories(grp_transactions, ['tr_drvr_code', 'tr_unit_id'])

#converting the truckID data type as category
grp_data_events.EquipmentID = grp_data_events.EquipmentID.astype('category')

trim_categories(grp_data_events, ['DriverID', 'EquipmentID'])

#merging the trasactions details and events dataframe
grp_trans_events = pd.merge(grp_transactions, grp_data_events,  how='left',left_on=['tr_drvr_code','tr_unit_id'],right_on=['DriverID','EquipmentID'])

#computing the total distance drives and mpg for drvr+truck combo
grp_trans_events['Total_dist_driven'] = grp_trans_events['odo_max'] - grp_trans_events['odo_min']
grp_trans_events['mpg_drvr_truck'] = grp_trans_events['Total_dist_driven']/grp_trans_events['Total_Fuel_Purchased']

#extracting the variables from performance dataset
grp_per = pd.DataFrame(performance_df.groupby(['DriverID','EquipmentID'])['TotalFuelUsed','Distance'].sum().add_prefix('Perf_')).reset_index()


grp_per.DriverID = grp_per.DriverID.astype('category')
grp_per.EquipmentID = grp_per.EquipmentID.astype('category')
trim_categories(grp_per, ['EquipmentID', 'DriverID'])

#final merge of transactions,events_fuel and performance df's
grp_merged  = pd.merge(grp_trans_events, grp_per,  how='left',left_on=['tr_drvr_code','tr_unit_id'],right_on=['DriverID','EquipmentID'])

#computing the difference for each truck and drvier's combo, the fuel purchased and the fuel burnt.... 
grp_merged['mpg_perf'] = grp_merged['Perf_Distance']/grp_merged['Perf_TotalFuelUsed']
grp_merged['diff_fuel_purchased'] = grp_merged['Total_Fuel_Purchased'] - grp_merged['Perf_TotalFuelUsed']

#converting the df into a csv file 
##grp_merged.sort_values('diff_fuel_purchased',ascending=False).to_csv('output.csv')

In [59]:

grp_merged[grp_merged.DriverID_x.isnull() == False].sort_values('diff_fuel_purchased',ascending=False).head(20)

,tr_drvr_code,tr_unit_id,Total_Fuel_Purchased,DriverID_x,EquipmentID_x,odo_max,odo_min,Total_dist_driven,mpg_drvr_truck,DriverID_y,EquipmentID_y,Perf_TotalFuelUsed,Perf_Distance,mpg_perf,diff_fuel_purchased
1685,WAGL,1534,14154.900391,WAGL,1534,501430.6,478171.4,23259.2,1.643191,WAGL,1534,3171.6,21266.0,6.705133,10983.300391
300,CHAD,2046,9721.559570,CHAD,2046,68778.3,18.2,68760.1,7.072950,CHAD,2046,325.5,2405.0,7.388633,9396.059570
256,BYRCHR,1734,23087.189453,BYRCHR,1734,534897.7,383534.9,151362.8,6.556138,BYRCHR,1734,15385.8,108548.0,7.055077,7701.389453
173,BOUB,1926,27916.279297,BOUB,1926,306093.7,92640.9,213452.8,7.646177,BOUB,1926,20690.9,148628.0,7.183254,7225.379297
1157,NANP,1899,22027.359375,NANP,1899,241628.6,57819.3,183809.3,8.344591,NANP,1899,14865.0,108378.0,7.290817,7162.359375
650,GRAJOH,1645,12789.889648,GRAJOH,1645,603582.6,516558.7,87023.9,6.804117,GRAJOH,1645,5717.3,39234.0,6.862330,7072.589648
168,BONZ,1919,26611.330078,BONZ,1919,281024.2,89987.2,191037.0,7.178784,BONZ,1919,19774.8,144467.0,7.305611,6836.530078
183,BRAE,1943,25469.990234,BRAE,1943,258473.7,85048.6,173425.1,6.808998,BRAE,1943,18718.3,133695.0,7.142476,6751.690234
1436,SAVE,1934,26680.240234,SAVE,1934,282397.8,91862.0,190535.8,7.141457,SAVE,1934,20314.9,148954.0,7.332254,6365.340234
687,HALCH,1873,26267.960938,HALCH,1873,223187.1,105160.0,118027.1,4.493196,HALCH,1873,19978.6,139521.0,6.983522,6289.360938


In [22]:
performance_df.columns

Index(['Id', 'EventTimeStamp', 'EquipmentID', 'DriverID', 'Latitude',
       'Longitude', 'LocationTimeStamp', 'IgnitionStatus', 'TripStatus',
       'DataStartTimeStamp', 'DataEndTimeStamp', 'Distance', 'DriveTime',
       'EngineTime', 'MoveTime', 'IntertripIdleTime', 'ShortIdleTime',
       'OverSpeedTime', 'TotalFuelUsed', 'IdleFuelUsed', 'FaultFlag',
       'RegisteredDriver', 'CruiseControlTime', 'TopGearTime',
       'GearDataSource', 'SpeedUnits'],
      dtype='object')

In [39]:
pd_cols = ['DriverID','EquipmentID', 'EventTimeStamp', 'DataStartTimeStamp',
          'DataEndTimeStamp', 'Distance', 'TotalFuelUsed', 'IdleFuelUsed'  ]
performance_df[performance_df.DriverID.isin(['CHAD'])][pd_cols]

,DriverID,EquipmentID,EventTimeStamp,DataStartTimeStamp,DataEndTimeStamp,Distance,TotalFuelUsed,IdleFuelUsed
76,CHAD,1656,2017-01-03 04:24:33,2016-12-27 04:22:00,2017-01-03 04:24:00,1558,214.6,2.3
770,CHAD,1656,2017-01-10 04:27:51,2017-01-03 04:24:00,2017-01-10 04:27:00,2623,387.8,8.6
1802,CHAD,1656,2017-01-17 05:00:02,2017-01-10 04:27:00,2017-01-17 04:59:00,1077,146.6,2.1
2331,CHAD,1656,2017-01-24 04:22:36,2017-01-17 04:59:00,2017-01-24 04:22:00,2081,287.0,2.8
2550,CHAD,1656,2017-01-31 04:13:13,2017-01-24 14:53:00,2017-01-24 16:24:00,7,1.3,0.3
2552,CHAD,1656,2017-01-31 04:13:14,2017-01-24 04:22:00,2017-01-31 04:13:00,2664,360.2,7.1
3368,CHAD,1656,2017-02-07 04:14:18,2017-01-31 04:13:00,2017-02-07 04:14:00,2114,307.0,4.6
3891,CHAD,1656,2017-02-14 04:15:46,2017-02-07 04:14:00,2017-02-14 04:15:00,3294,469.7,13.2
4767,CHAD,1656,2017-02-21 04:09:41,2017-02-14 04:15:00,2017-02-21 04:09:00,2557,357.6,3.8
4999,CHAD,1656,2017-02-28 04:20:26,2017-02-21 04:09:00,2017-02-28 04:20:00,3098,424.8,3.1


In [25]:
events_levels_df.columns

Index(['index', 'Id', 'EventTimeStamp', 'EquipmentID', 'DriverID', 'Latitude',
       'Longitude', 'LocationTimeStamp', 'Speed', 'Odometer', 'IgnitionStatus',
       'EFReportReason', 'TankLevelPercent', 'TankLevelGallons',
       'ExactFuelEventId', 'truck', 'tankcap', 'refuel_datetime',
       'refuel_tr_sum', 'refuel_tr_id'],
      dtype='object')

In [40]:
el_cols = ['DriverID', 'EquipmentID', 'EventTimeStamp', 'Odometer', 'Speed'   ]
el_bs = np.logical_and(events_levels_df.DriverID.isin(['CHAD']), 
                      events_levels_df.EquipmentID.isin(['1656']))
events_levels_df[el_bs][events_levels_df.EventTimeStamp.gt('2017-01-01')][el_cols]
#events_levels_df.EventTimeStamp.gt

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,DriverID,EquipmentID,EventTimeStamp,Odometer,Speed
1094824,CHAD,1656,2017-01-02 15:10:46.500,431725.8,0
1094825,CHAD,1656,2017-01-02 15:20:06.607,431725.8,0
1094826,CHAD,1656,2017-01-03 03:01:45.210,431725.8,0
1094827,CHAD,1656,2017-01-03 03:16:45.197,431730.1,62
1094828,CHAD,1656,2017-01-03 04:16:45.663,431793.2,65
1094829,CHAD,1656,2017-01-03 04:54:16.180,431832.0,11
1094830,CHAD,1656,2017-01-03 05:54:16.790,431897.6,66
1094831,CHAD,1656,2017-01-03 06:10:07.070,431913.5,4
1094832,CHAD,1656,2017-01-03 06:25:17.070,431921.7,40
1094833,CHAD,1656,2017-01-03 06:40:17.210,431934.3,65


In [11]:
events_grp =events_levels_df[['DriverID', 'EventTimeStamp', 'EquipmentID',
                              'Odometer', 'refuel_tr_id', 'refuel_tr_sum', 
                              'refuel_datetime']].groupby(['DriverID', 'EquipmentID', 'refuel_datetime'])

agg_dict = {'Odometer' : ['min', 'max'], 
           'EventTimeStamp' : ['first', 'last'], 
           'refuel_tr_id' : 'first', 
           'refuel_tr_sum' : 'first'}
events_grp_df = events_grp.agg(agg_dict)

events_grp_df.info(memory_usage='deep', null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 101113 entries, (ABBV, 1821, 2017-03-04 13:49:00) to (ZAMJ, 1588, 2017-06-01 08:14:00)
Data columns (total 6 columns):
(Odometer, min)            101113 non-null float64
(Odometer, max)            101113 non-null float64
(EventTimeStamp, first)    101113 non-null datetime64[ns]
(EventTimeStamp, last)     101113 non-null datetime64[ns]
(refuel_tr_id, first)      101079 non-null object
(refuel_tr_sum, first)     101113 non-null float32
dtypes: datetime64[ns](2), float32(1), float64(2), object(1)
memory usage: 11.3 MB


In [16]:
events_levels_df.truck.value_counts()

1918    33866
2003    25719
310     24865
1890    24479
1808    24097
1957    23578
1865    23262
1666    23026
1995    22756
1879    22580
1843    22235
1715    22207
1941    22103
1814    22053
1847    22036
1994    21876
1792    21484
1721    21084
1732    20982
1684    20952
1772    20924
1972    20753
1938    20500
2014    20314
1729    20193
1848    20007
1934    19902
1824    19883
1734    19868
1704    19787
        ...  
1569     1481
2136     1357
2134     1178
301      1108
1614     1082
1622      911
1525      788
1548      468
1625      355
2151      250
2150      239
2155      238
2147      228
2153      226
2152      157
2144      154
1627      148
2146      146
2154      142
2148      130
1628      106
1513       75
1635       28
1617       18
1621       16
1508       15
1636        7
1516        4
2156        2
1519        1
Name: truck, Length: 629, dtype: int64

In [14]:
events_grp_df.index.get_level_values(0).value_counts()

VAUJAM     348
ECKS       339
CLIJ       329
CHRN       326
ARCM       301
WILANT     301
BLACH      300
ROBJEF     299
AMMC       297
CRAC       293
WILMIC     289
DEASA      288
GALJU      286
CARDE      283
DOBE       283
BROD03     279
KORR       276
SAVE       275
BYRCHR     275
LONRO      274
GRIST      272
BOUB       272
CARRO      272
JONCH      271
AUSTE      271
TUSR       268
ULAL       267
COTALV     267
SIMJIM     265
RODMAR     264
          ... 
MORTIM       2
HAYM         1
ADADE        0
WILSO3       0
ACHC         0
CARJ01       0
WILRON       0
TAYDO1       0
CRAIG B      0
BYRCHI       0
HEAD         0
STAW         0
FURLA        0
HOLW1        0
HARGR        0
KINCA        0
KUNDAV       0
GODJO        0
BEAT         0
MEID         0
BANJA        0
SNID         0
DENJE        0
RUDD         0
SCHJ         0
DAVKEN       0
SHOP1        0
SHOP2        0
SHOP3        0
36728        0
Name: DriverID, Length: 924, dtype: int64